In [ ]:
!pip install kagglehub

import kagglehub

# Descargar dataset
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

# EDA

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Rutas del dataset
base_dir = path + "/chest_xray"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
val_dir = os.path.join(base_dir, "val")

# Contar imágenes por clase
def contar_imagenes(directorio):
    clases = ['NORMAL', 'PNEUMONIA']
    conteo = {}
    for clase in clases:
        conteo[clase] = len(os.listdir(os.path.join(directorio, clase)))
    return conteo

train_counts = contar_imagenes(train_dir)
test_counts = contar_imagenes(test_dir)
val_counts = contar_imagenes(val_dir)

# Mostrar conteos
print("Train:", train_counts)
print("Test:", test_counts)
print("Val:", val_counts)

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=list(train_counts.keys()), y=list(train_counts.values()), palette="viridis")
plt.title("Distribución de clases en conjunto de entrenamiento")
plt.ylabel("Cantidad de imágenes")
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Ruta base 
dataset = os.path.join(path, "chest_xray/train")

# Clases 
classes = os.listdir(dataset)
print("Clases encontradas:", classes)

# Mostrar una imagen por clase
plt.figure(figsize=(10, 5))

for i, class_name in enumerate(classes):
    class_path = os.path.join(dataset, class_name)
    # Verificamos que existan imágenes
    images = os.listdir(class_path)
    if len(images) == 0:
        print(f"Carpeta vacía: {class_name}")
        continue
    img_path = os.path.join(class_path, images[0])
    img = Image.open(img_path)

    plt.subplot(1, len(classes), i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(class_name)
    plt.axis('off')

plt.suptitle("Ejemplos de imágenes por clase", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# ====== LIBRERÍAS ======
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ====== RUTA DEL DATASET ======
base_dir = os.path.join(path, "chest_xray")

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

# ====== GENERADORES DE IMÁGENES ======
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=8,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=8,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=8,
    class_mode='binary'
)

# ====== MODELO CNN SENCILLO ======
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# ====== COMPILACIÓN ======
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ====== ENTRENAMIENTO ======
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# ====== EVALUACIÓN ======
test_loss, test_acc = model.evaluate(test_gen)
print(f"\n✅ Precisión en test: {test_acc:.4f}")

# ====== GRÁFICOS ======
plt.figure(figsize=(10,4))
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Evolución de la Precisión')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# ====== GUARDAR MODELO ======
model.save("modelo_neumonia_keras.h5")

